In [52]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

pdf_files = ["documents/pruebaLLM.pdf","documents/pruebaLLM2.pdf", "documents/MartinScorsese.pdf"]  # Agrega la lista de archivos PDF
# pdf_files = ["documents/pruebaLLM2.pdf"]  # Agrega la lista de archivos PDF

all_documents = []

for document in pdf_files:
    loader = PyPDFLoader(document)
    data = loader.load()
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=400, chunk_overlap=20)    # Split the data into chunks
    documents = text_splitter.split_documents(data)
    all_documents.extend(documents)

print(f"Se cargaron y dividieron {len(all_documents)} fragmentos de texto.")



Se cargaron y dividieron 220 fragmentos de texto.


In [53]:
from langchain_huggingface import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

In [54]:
from langchain_chroma import Chroma

# Crear un vector store de Chroma usando los documentos y el modelo de embeddings
vector_store = Chroma.from_documents(
    documents=all_documents,
    collection_name="pdfs_2",
    embedding=embeddings,  # Pasar el modelo de embeddings aquí
)

print(f"Se creó el vector store con {len(all_documents)} documentos.")

Se creó el vector store con 220 documentos.


In [55]:
from langchain_chroma import Chroma

vector_store = Chroma(
    collection_name="pdfs_2",
    embedding_function=embeddings,
)

In [56]:
retriever = vector_store.as_retriever()

In [57]:
question = "Como es la filmografia de Martin Scorsese"
docs = vector_store.similarity_search(question)
len(docs)

4

In [58]:
docs[0]

Document(metadata={'page': 21, 'source': 'documents/MartinScorsese.pdf'}, page_content='500. Paris, 1996.\nMarino, Giuseppe Cario. Historiade laMafia. Barcelona: Ediciones B, 2002.\nMillas, Jorge. LaViolenciay sus Máscaras. Santiago: Aconcagua, 1976.\nRuiz, Raúl. PoéticaDel Cine. Santiago: Sudamericana, 2000.\nScorsese, Martínyotros. APersonalJourneyWithMartínScorseseThroughAmericanMovie.\nLondon: Akal, 1997.\nScorsese, Martínyotros. Conversaciones con Scorsese. Madrid: Plot, 1987.')

In [59]:
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts import ChatPromptTemplate
from langchain_ollama.chat_models import ChatOllama
from langchain_core.runnables import RunnableLambda, RunnablePassthrough

# Prompt
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

# Local LLM
ollama_llm = "llama3.2"
model_local = ChatOllama(model=ollama_llm)

# Chain
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model_local
    | StrOutputParser()
)

In [65]:
chain.invoke("pelicula y color favorito de Juan(pruebaLLM2.pdf)")

'La respuesta a la pregunta es:\n\n- Película favorita de Juan: "Mulholland Drive"\n- Color favorito de Juan: Amarillo'

In [61]:
chain.invoke("Quien es Martin Scorsese")

'Martin Scorsese es un cineasta conocido.'

In [62]:
chain.invoke("Que relación tiene la violencia es sus películas?")

'La violencia tiene una relación profunda y central en las películas de Martin Scorsese. Según el texto, la violencia es un aspecto inherente al hombre y a la sociedad, que aparece en muchas de sus obras, como en "Mean Streets", "Casino" (1995) entre otras. La violencia es presentada como una forma de vida, una opción creativa que se explora en las películas de Scorsese.\n\nEn este sentido, la violencia no solo se muestra en la acción física o en los conflictos, sino que también se refleja en la psicología y el comportamiento de los personajes. La violencia es un tema que se aborda de manera sutil y elíptica, lo que le da un carácter trágico y profundo.\n\nEn resumen, la violencia es una constante en las películas de Martin Scorsese, y se considera un aspecto fundamental de su estilo y visión artística.'

In [63]:
chain.invoke("Háblame de Goodfellas")

'Según el contexto proporcionado, "Goodfellas" es un film dirigido por Martin Scorsese que explora la evocación de la elegancia del gángster en la tradición del cine de género. El filme está basado en la vida real de Henry Hill, quien se convirtió en un testigo estrella para las autoridades después de ser capturado y confesar sus crímenes.\n\nEn el contexto del documental "AFI\'S 100 Years... Loomovies" realizado por Scorsese en 1998, se menciona que el film está dentro de la tradición del cine negro norteamericano, caracterizado por la puesta en escena de un universo de gángsters y delincuentes.\n\nEl filme también parece estar relacionado con otras obras de Scorsese, como "Mean Streets" (1973), "Casino" (1995) y "Raging Bully", que comparten una estructura y una disposición similar. Además, se menciona la circularidad en las escenas del film, que también se repite en otras películas de Scorsese.\n\nEn resumen, "Goodfellas" parece ser un filme dirigido por Martin Scorsese que explora 